In [2]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pandas as pd 
import re
import numpy as np
from gensim.models import Doc2Vec, doc2vec
#import numpy as np
from nltk.corpus import stopwords
from textblob import TextBlob, Word
import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')
import pandas as pd


Using TensorFlow backend.


In [3]:



df = pd.read_csv('train.csv',dtype={"label": object, "text": object})
#df = pd.read_csv("sample.csv", dtype={"label": object, "text": object})
y = df.label 
text = df.text

#print(df)
#Convert all text to lower case
df.text = df.text.apply(lambda x: " ".join(x.lower() for x in str(x).split()))

#Remove all punctuation
for i in range(0, len(text)):
    df.text[i] = re.sub(r'[^\w\s]', "", df.text[i])
    
stop = stopwords.words("english")
# for i in range(0, len(text)):
#   line = text[i].split()
#   for word in line:
#     if word in stop:
#       line.remove(word)
#   text[i] = " ".join(line)
      
#Remove stop words
df.text = df.text.apply(lambda x: " ".join(x for x in str(x).split() if x not in stop))

#Lem
df.text = df.text.apply(lambda x: " ".join(Word(x).lemmatize() for x in x.split()))

for i, line in enumerate(df.text):
    df.text[i] = line.split()

print(df)

       label                                               text
0          1  [great, mobile, app, nice, reward, program, ma...
1          2  [really, fast, polite, definitely, recommend, ...
2          2  [place, always, amazing, friendly, staff, grea...
3          1  [wine, 101, class, friday, night, cool, spot, ...
4          1  [rounding, think, place, may, potential, coupl...
5          0  [romantic, tryst, planet, hollywood, poker, ro...
6          2  [got, color, haircut, finally, month, looking,...
7          0  [never, go, place, nearly, empty, 430, pm, sat...
8          2  [ive, come, 2, year, np, bought, staff, always...
9          0  [07082014, war, ich, mit, 5, kollegen, mittags...
10         2  [5, star, takeout, recommended, dinein, liquor...
11         1  [nachdem, wir, jetzt, da, dritte, mal, vor, de...
12         2  [hand, best, polish, boy, sandwich, cleveland,...
13         2  [oh, fuzzys, begin, familiar, toby, keith, son...
14         2  [ive, tried, many, cupcake

In [9]:
del(data)
data = df[['text', 'label']]


In [11]:
data.text


0         [great, mobile, app, nice, reward, program, ma...
1         [really, fast, polite, definitely, recommend, ...
2         [place, always, amazing, friendly, staff, grea...
3         [wine, 101, class, friday, night, cool, spot, ...
4         [rounding, think, place, may, potential, coupl...
5         [romantic, tryst, planet, hollywood, poker, ro...
6         [got, color, haircut, finally, month, looking,...
7         [never, go, place, nearly, empty, 430, pm, sat...
8         [ive, come, 2, year, np, bought, staff, always...
9         [07082014, war, ich, mit, 5, kollegen, mittags...
10        [5, star, takeout, recommended, dinein, liquor...
11        [nachdem, wir, jetzt, da, dritte, mal, vor, de...
12        [hand, best, polish, boy, sandwich, cleveland,...
13        [oh, fuzzys, begin, familiar, toby, keith, son...
14        [ive, tried, many, cupcake, magnolia, prarie, ...
15        [put, irish, pub, walmart, youd, get, dont, ge...
16        [amazing, pho, almost, drinkin

In [5]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

AttributeError: 'list' object has no attribute 'replace'

In [26]:
vocabulary=[]
data['text'][0]
for key, value in data['text'].items():
    temp = value
    vocabulary.append(temp)
vocabulary
vocabulary= [x.encode('ascii')  for x in vocabulary]

AttributeError: 'list' object has no attribute 'encode'

In [10]:
data['text']= [x.encode('ascii') for x in data['text']]

AttributeError: 'list' object has no attribute 'encode'

In [31]:
tokenizer = Tokenizer(nb_words=2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')
tokenizer.fit_on_texts(data['text'].values)
tokenizer
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

/home/saura/anaconda3/lib/python3.6/site-packages/keras/preprocessing/text.py:172: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [46]:
embed_dim = 128
lstm_out = 196
batch_size = 32

model = Sequential()
model.add(Embedding(2000, embed_dim,input_length = X.shape[1], dropout = 0.2))
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(units=4,activation='relu'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


/home/saura/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
/home/saura/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  import sys


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 486, 128)          256000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 788       
Total params: 511,588
Trainable params: 511,588
Non-trainable params: 0
_________________________________________________________________
None


In [47]:
X

array([[   0,    0,    0, ...,  411,   40, 1296],
       [   0,    0,    0, ...,  115,    9, 1375],
       [   0,    0,    0, ...,  312,    3,  196],
       ..., 
       [   0,    0,    0, ...,  277,    3,    1],
       [   0,    0,    0, ...,    1,    7,  113],
       [   0,    0,    0, ...,   68,  252,   11]], dtype=int32)

In [48]:
X.shape

(400001, 486)

In [49]:
Y = pd.get_dummies(data['label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(320000, 486) (320000, 4)
(80001, 486) (80001, 4)


In [ ]:
model.fit(X_train, Y_train, batch_size = batch_size, nb_epoch = 1, verbose = 2)



/home/saura/anaconda3/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
